In [1]:
## import data processing/cleaning , data modeling libraries
import pandas as pd
import os
import sys
import re as re
import datetime as datetime
import numpy as np
import collections
import string
import pandas as pd

from nltk.tag.perceptron import PerceptronTagger
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords

from gensim.models import doc2vec
from gensim.models import Word2Vec
from gensim.models import fasttext

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from gensim.models.wrappers import FastText
from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
# from sklearn.tree import DecisionTreeRegressor
# from xgboost.sklearn import XGBClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')
t0 = datetime.datetime.now()

/home/ubuntu/anaconda3/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
outData43 = pd.DataFrame()
limit = 20000

outData43_text = pd.read_csv("reviews.txt",encoding="utf-8",header=None,names=["text"])
outData43_lab = pd.read_csv("labels.txt",encoding="utf-8",header=None,names=["label"])
#outData43_text = outData43_text[:limit]
#outData43_lab = outData43_lab[:limit]
#outData43["length"]=outData43_text.text.apply(lambda x:len(x))
# outData43["label"]=outData43_lab.label
#print(outData43.length)
# print(outData43.head(n=2))

x_train,x_test,y_train,y_test = train_test_split(outData43_text,outData43_lab,test_size=0.2)

In [3]:
## NLP PRE-PROCESSING

#text=" the JanaSena Party Formation Day celebrations || LIVE funny celebration|| Pawan Kalyan || Guntur"
text="Chak De India | Full Title Song | Shah Rukh Khan | Sukhvinder Singh | Salim | Marianne D'Cruz"

def nltk_clean_sent(line):
    if len(line)>0:
        ## remove the punctuation/emoticons/digits/multispaces with single from the line
        ## dont make lowercase before the pos tagging
        line_lower = line.strip()
        line_punct = re.sub('['+string.punctuation+']',' ',line_lower)
        line_emots = re.sub(r'[\u200b-\u2fff]+',' ',line_punct)
        line_digis = re.sub(r'[0-9]+',' ',line_emots)
        line_spaces = re.sub(r'[\s]+',' ',line_digis)
        line = line_spaces
    return line.lower()

def nltk_extract_postags(tokens):
    cleaned_str = ''
    #tokens = nltk.word_tokenize(line)
    tokens_pos = PerceptronTagger().tag(tokens)
    #print(tokens_pos)
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    nltk_tags = nn_tags + jj_tags + vb_tags
    
    for tag_word in tokens_pos:
#         if tag_word[1] in nltk_tags:
            if len(tag_word[0])>2:
                cleaned_str += tag_word[0].lower()+tag_word[1]+' '
    return cleaned_str.strip().split()

def nltk_apply_lemma(line):
    ## lemma is fine rather than stemming
    tokens_lemmas = [WordNetLemmatizer().lemmatize(word) for word in line.split()]
    #tokens_lemmas = [PorterStemmer().stem(word) for word in line.split()]
    ## stemming
    tokens_stops = [word for word in tokens_lemmas if word not in stopwords.words('english')]
    tokens_stops = [word for word in tokens_stops if len(word.strip())>2]
    tokens_stops = list(set(tokens_stops))
    return tokens_stops

def nltk_extract_tags(line):
        ## tokenize the sentence/get tokens that contains only letters
        line_clean = nltk_clean_sent(line)
        ## apply lemmatize/stemming and remove stopwords
        token_lemma = nltk_apply_lemma(line_clean)
#         apply postags to the words and get only couple of tags and word length >2
#         tokens_pos = nltk_extract_postags(token_lemma)
        
        return token_lemma
    
nltk_extract_tags(text)

['cruz',
 'salim',
 'india',
 'title',
 'marianne',
 'chak',
 'rukh',
 'khan',
 'song',
 'singh',
 'full',
 'shah',
 'sukhvinder']

In [4]:
# Compute clean title
x_train['clean_title_words'] = x_train.text.apply(nltk_extract_tags)
x_test['clean_title_words'] = x_test.text.apply(nltk_extract_tags)
#print(df_en.head(n=2))
x_train['clean_title_text'] = x_train.clean_title_words.apply(lambda x:' '.join(x))
x_test['clean_title_text'] = x_test.clean_title_words.apply(lambda x:' '.join(x))
print(x_train.head(n=2))

                                                   text  \
4726  the story is somewhat stilted  what with the m...   
8368  based on the idea from gackt  moon child took ...   

                                      clean_title_words  \
4726  [main, memorable, sally, writer, ever, compete...   
8368  [main, become, considering, stricken, order, m...   

                                       clean_title_text  
4726  main memorable sally writer ever compete affec...  
8368  main become considering stricken order made gr...  


In [9]:
# ## DOC2VEC model
#finaldoc = list(x_train['clean_title_words'])+list(x_test['clean_title_words'])
cleaneddoc = list(x_train['clean_title_words'])
cleaneddoc1 = list(x_test['clean_title_words'])
print(cleaneddoc[:1])
taggeddoc = [doc2vec.TaggedDocument(val,[idx]) for idx,val in enumerate(cleaneddoc)]

d2vmodel = doc2vec.Doc2Vec(size=50,min_count=2,epochs=10)
d2vmodel.build_vocab(taggeddoc)
# #print(d2vmodel[0])
d2vmodel.train(taggeddoc,total_examples=d2vmodel.corpus_count,epochs=d2vmodel.epochs)

xtrain = [d2vmodel.infer_vector(val) for idx,val in enumerate(cleaneddoc)]
xtest = [d2vmodel.infer_vector(val) for idx,val in enumerate(cleaneddoc1)]
print(xtest[0])

##feature_vecto = d2vmodel.docvecs.doctag_syn0
##print(len(d2vmodel_vecs))

[['brown', 'performance', 'year', 'give', 'dream', 'case', 'awesome', 'fricker', 'find', 'movie', 'daniel', 'much', 'forty', 'palsey', 'someone', 'reason', 'christie', 'day', 'severe', 'lived', 'cerebral', 'inspirational', 'brenda', 'lewis', 'achieve']]
[ -1.52037768e-02  -1.73712745e-01   8.37332606e-02  -1.06364302e-01
   4.58639525e-02  -4.84243445e-02  -1.76045910e-01  -2.00325564e-01
   3.11818480e-01   8.02127831e-03   2.00823903e-01  -2.56282508e-01
   1.08550526e-01   1.03271924e-01   1.45190060e-01  -3.85168642e-02
   3.04789603e-01  -1.47695690e-02   4.89773974e-02   7.04371631e-02
  -5.80405928e-02  -1.17852710e-01   1.46775588e-01   1.28019154e-01
   1.15412660e-01   4.31235224e-01   2.73393095e-02  -1.29886091e-01
   7.83788934e-02  -3.56101662e-01   1.11137748e-01   1.43128023e-01
   3.12396973e-01  -3.22554976e-01   2.78450698e-01   3.15884918e-01
  -1.81622338e-02   4.52530151e-03   9.94418189e-02   3.94219905e-01
  -3.57410114e-04  -4.24078256e-02   1.84804127e-01   1.

20000
5000


training accuracy score= 0.834
             precision    recall  f1-score   support

   negative       0.82      0.85      0.83      2458
   positive       0.85      0.82      0.83      2542

avg / total       0.83      0.83      0.83      5000

TIME taken for Base Modeling (LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.1,
     verbose=0) with Universal Sentence Encoder)= 0:00:04.955837
training accuracy score= 0.826
             precision    recall  f1-score   support

   negative       0.82      0.83      0.83      2490
   positive       0.83      0.82      0.83      2510

avg / total       0.83      0.83      0.83      5000

TIME taken for Base Modeling (SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=50, n_